In [6]:
import json
import re

from pathlib import Path
from typing import Any, Dict, List, Set, Union

import nltk
import numpy as np
import pandas as pd
import requests

from PyDictionary import PyDictionary
from tqdm.notebook import tqdm

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Initialize Basic Word List

In [8]:
def load_basic_word_list(stopwords: Union[List[str], Set[str]] = nltk.corpus.stopwords.words('english')) -> Set[str]:
    """Loads the Ogden Basic English Word List and merges it with the Wikipedia Basic Word List to create a set
    of basic English words, filtering out all stopwords.
    
    Requires the files ogden_basic_words.txt and wikipedia_words.txt stored in the same directory under the subdirectory 
    /data (e.g. current_directory/data/ogden_basic_words.txt).
    
    Parameters
    ----------
    stopwords: Union[List[str], Set[str]]
        Words to filter out of the basic word list
    
    Returns
    ----------
    Set[str]
        All basic English words
    """
    stopwords = set(stopwords)  # Ensure set to leverage hashing
    
    def load_ogden_basic_words(path: str) -> Set[str]:
        with open(path, 'r') as file:
            raw_words = file.read().replace('\n', ',').lower().split(',')
            
        words = []
        for raw_word in raw_words:
            word = raw_word
            for value in ['basic:', 'intl:', 'endings:', 'compound:', 'next:', '(', ')', '.']:
                word = word.replace(value, '')
            words.append(word.strip())
        return set(word for word in words if word)
            
    def load_wikipedia_words(path: str) -> Set[str]:
        with open(path, 'r') as file:
            raw_words = file.read().replace('\n', '').replace('.', '').lower().split(',')
        return set(word.strip() for word in raw_words if word)
    
    basic_word_list: set = set([])
    word_list_directory = Path().resolve().parents[0] / 'data' / 'basic_word_lists'
    basic_word_lists: Dict[str, str] = {
        'ogden_basic_words.txt': load_ogden_basic_words,
        'wikipedia_words.txt': load_wikipedia_words
    }
    for filename, read_function in basic_word_lists.items():
        basic_word_list |= read_function(word_list_directory / filename)
    
    return set(word for word in basic_word_list if word not in stopwords)

In [ ]:
basic_word_list = load_basic_word_list()

## Token and Tokenizer Classes

In [9]:
class Token:
    
    def __init__(self, token: str):
        self.string: str = token.strip()
        self.dictionary: PyDictionary = PyDictionary(self.string)
        self.complexity_score: int = 0
            
    def initialize_all_attributes(self, basic_word_list: Union[list, set]) -> bool:
        """Initializes all class attributes to avoid manually running down the list each time.
        
        Parameters
        ----------
        basic_word_list: Union[List[str], Set[str]]
            the set of "Basic" words for self.semantic_word_complexity
    
        Returns
        ----------
        bool
            successfully initialized everything?
        """
        self.get_semantic_word_complexity(basic_word_list)
        return True
        
    def get_semantic_word_complexity(self, basic_word_list: Union[List[str], Set[str]]) -> int:
        """Follows the algorithm proposed by the research paper "Estimation of Lexical Complexity using Language Semantics"
        by Nimavat and Champaneria. https://www.researchgate.net/publication/320466030
        
        :param Union[List[str], Set[str]] basic_word_list: the set of "Basic" words according to the proposed algorithm
        :return int: the Semantic Word Complexity Score
        """
        if self.complexity_score:
            return self.complexity_score
        
        stopwords = nltk.corpus.stopwords.words('english')
        tokens = self.get_definition_as_tokens(self.string, stopwords)
        for token in tokens:
            if token in basic_word_list:
                self.complexity_score += 1
            else:
                tokens2 = self.get_definition_as_tokens(token, stopwords)
                self.complexity_score += len(tokens2)
        return self.complexity_score
            
    def get_definition_as_tokens(self, word: str, stopwords: Union[List[str], Set[str]] = []) -> List[str]:
        """Converts the first definition for the provided word from PyDictionary, filtering our the provided stopwords.
        
        :param str word: the word to look up the definition of
        :param Union[list, set] stopwords: the words to remove from the definition 
        :return List[str]: list of all alphabetical, non-stopword tokens lowercase
        """
        tokens = []
        if definitions := self.dictionary.meaning(word):
            for pos, pos_definitions in definitions.items():
                for definition in (pos_definitions or []):
                    raw_tokens = nltk.word_tokenize(re.sub(r'\((.*?)$', '', definition))
                    tokens.extend([t.lower() for t in raw_tokens if t.lower() not in stopwords and t.isalpha()])
                    if tokens:
                        return tokens
        return tokens
    
    def to_dict(self) -> Dict[str, Dict[str, Any]]:
        """Converts all basic-type attributes from this class into a nested dictionary following the pattern:
        {'self.string': {'complexity_score': 'self.complexity_score', ...}}
        
        :return Dict[str, Any]: dictionary with all class attribute names as keys and values and values
        """
        allowed_types = (str, int, float, list, dict, set, tuple)
        as_dict = {key: value for key, value in vars(self).items() if isinstance(value, allowed_types)}
        return {as_dict.pop('string'): as_dict}
    
    def from_dict(self, as_dict: Dict[str, Dict[str, Any]]) -> bool:
        """Converts a dictionary into this class's attributes to prevent recalculate metrics every time.
        as_dict should follow the following pattern:
        {'self.string': {'complexity_score': 'self.complexity_score', ...}}
        
        :return bool: successfully initialized self attributes?
        """
        class_attributes = vars(self)
        for string, attributes in as_dict.items():
            if self.string != string.strip():
                continue
            for attribute_name, attribute_value in attributes.items():
                if attribute_name in class_attributes:
                    self.__setattr__(attribute_name, attribute_value)
            break
        else:
            raise Exception(f"The passed dictionary (as_dict) has no key in it matching self.string: {self.string}")
        return True
    

class TokenFeaturizer:
    
    def __init__(self, path: Union[str, None] = None):
        self.path: Union[str, Path] = path or Path().resolve() / 'all_tokens.json'
        self.all_tokens: Dict[str, Dict[str, str]] = self.load_all_tokens()
        self.average_complexity_score: float = 0.0
        self.basic_word_list: Union[List[str], Set[str]] = load_basic_word_list()
        
    def get_token_object(self, word: str) -> Token:
        """If the word is already in self.all_tokens data, will initialize a new Token object using this data and the
        Token.from_dict() method. Otherwise, create a new Token instance, initialize its attributes, and store it in
        self.all_tokens to avoid reinitialization in the future.
        
        :param str word: the word to create a Token from
        :return Token: an instance of the Token class initialized on the parameter word
        """
        token = Token(word)
        if token_data := self.all_tokens.get(word):  # Word already in all_tokens
            if all([key in token_data for key in token.to_dict().get(word, {})]):  # No new attributes in Token class
                processed_word = word.strip()
                token.from_dict({processed_word: token_data})
                return token
            
        token.initialize_all_attributes(self.basic_word_list)
        self.all_tokens.update(token.to_dict())
        return token
    
    def get_average_complexity_score(self) -> float:
        """Averages the complexity score of all non-zero complexity tokens in self.all_tokens.
        
        :return float: the average complexity score (excluding tokens with complexity score of zero)
        """
        if not self.average_complexity_score:
            non_zero_tokens = 0
            total_complexity_score = 0
            for token, token_data in self.all_tokens.items():
                if complexity_score := token_data.get('complexity_score'):
                    non_zero_tokens += 1
                    total_complexity_score += complexity_score
            self.average_complexity_score = np.round(total_complexity_score / non_zero_tokens, 2)
        return self.average_complexity_score
            
    def load_all_tokens(self) -> Dict[str, Dict[str, Any]]:
        """Given the proper path to the all_tokens.json file, convert the file into a dictionary for faster look up.
        This dictionary prevents you from recalculating the attributes of all known tokens.

        :return Dict[str, Dict[str, Any]]: token name as a key with its associated metrics 
        """
        if not self.path.is_file():
            return {}
        with open(self.path, 'r') as file:
            return json.load(file)
        
    def save_all_tokens(self) -> Dict[str, Dict[str, Any]]:
        """Given the proper path to the all_tokens.json file, convert the file into a dictionary for faster look up.
        This dictionary prevents you from recalculating the attributes of all known tokens.

        :return Dict[str, Dict[str, Any]]: token name as a key with its associated metrics 
        """
        with open(self.path, 'w') as file:
            json.dump(self.all_tokens, file, indent=4)

In [ ]:
all_tokens_json_path = Path().resolve().parents[0] / 'data' / 'all_tokens.json'
tf = TokenFeaturizer(all_tokens_json_path)

## Process Training DataFrame

### Clean the DataFrame

In [ ]:
def remove_lrb_rrb_tags(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Removes -LRB-...-RRB- tags along with all text between from 'original_text' column.
    
    Expected dataframe columns:
        original_text: str
    
    :return pd.DataFrame: the passed DataFrame cleaned
    """
    dataframe.original_text = dataframe.original_text.str.replace(r' -LRB-(.*?)-RRB-', '', regex=True)
    return dataframe

def replace_html_char_codes_with_char(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Replaces char codes (e.g. &ndash;) with their respective characters.
    
    Expected dataframe columns:
        original_text: str
    
    :return pd.DataFrame: the passed DataFrame cleaned
    """ # TODO: Find/compile CSV of all codes and dynamically import and replace all
    characters: Dict[str, str] = { # Code: Character
        '&ndash;': '–',
        '&mdash;': '—',
    }
    for code, character in characters.items():
        dataframe.original_text = dataframe.original_text.str.replace(f"{code[0]} {code[1:-1]} {code[-1]}", character)
    return dataframe

def clean_dataset(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Performs all required data cleaning on all records in the passed DataFrame.
    
    Expected dataframe columns:
        original_text: str
        label: int (0 or 1)
    
    :return pd.DataFrame: the passed DataFrame cleaned
    """
    cleaning_steps = [
        remove_lrb_rrb_tags,
        replace_html_char_codes_with_char,
    ]
    for cleaning in cleaning_steps:
        dataframe = cleaning(dataframe)
    return dataframe

In [ ]:
training_path = Path().resolve().parents[0] / 'data' / 'WikiLarge_Train.csv'
df = pd.read_csv(training_path)
df = clean_dataset(df)
df.head()

In [ ]:
testing_path = Path().resolve().parents[0] / 'data' / 'WikiLarge_Test.csv'
df2 = pd.read_csv(testing_path)
df2 = clean_dataset(df2)
df2.head()

### Initialize All Complexity Score Features

In [ ]:
def get_tokenized_sentence(sentence: str, 
                           stopwords: Union[List[str], Set[str]] = nltk.corpus.stopwords.words('english')) -> List[str]:
    """Convert the sentence str into a list of tokens, filtering our the provided stopwords.
        
    :param str sentence: the sentence string to convert
    :param Union[List[str], Set[str]] stopwords: the words to remove from the definition 
    :return List[str]: list of all alphabetical, non-stopword tokens lowercase
    """
    tokens = nltk.word_tokenize(sentence)
    return [token.lower() for token in tokens if token.lower() not in stopwords and token.isalpha()]

def get_complexity_scores(tf: TokenFeaturizer, sentence: str) -> List[Union[int, float]]:
    """Calculates a list of complexity scores for all valid tokens in sentence. Will replace all 0 complexity scores with
    the average complexity score of all_tokens stored in the param tf.
    
    :param TokenFeaturizer tf: an initialized TokenFeaturizer instance
    :param str sentence: the sentence to calculate the complexity score of
    :return List[Union[int, float]]: all complexity scores for non-stopword tokens in sentence
    """
    return [cs if (cs := tf.get_token_object(tok).complexity_score) else tf.get_average_complexity_score()
            for tok in get_tokenized_sentence(sentence)]

def process_features(df: pd.DataFrame, all_tokens_path: Union[Path, str]) -> pd.DataFrame:
    """Adds the following columns to the parameter DataFrame: [
        'text_length': int,  # length of all tokens in original_text
        'token_count': int,  # all non-stopword tokens
        'stopword_count': int,
        'complexity_scores': List[float],  # complexity scores of all non-stopword tokens (zeros replaced by average)
        'sum_cs': float,
        'avg_cs': float,
    ]
    
    :param pd.DataFrame df: required column: 'original_text': str
    :return pd.DataFrame: the original dataframe with more feature columns
    """
    tf = TokenFeaturizer(all_tokens_path)
    df['text_length'] = df['original_text'].str.count(' ') + 1
    df['complexity_scores'] = df['original_text'].apply(lambda x: get_complexity_scores(tf, x))
    df['token_count'] = df['complexity_scores'].str.len()
    df['stopword_count'] = df['text_length'] - df['token_count']
    df['sum_cs'] = df['complexity_scores'].map(sum)
    df['avg_cs'] = df['sum_cs'] / df['token_count']
    return df

In [ ]:
df = process_features(df, all_tokens_json_path)
df.head()